In [13]:
from datasets import load_dataset
import ollama
dataset = load_dataset("ag_news")
texts = dataset["train"]["text"]
labels = dataset["train"]["label"]
mapping = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
print("texts:", texts[:5])
print("labels:", labels[:5])

texts: ["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.', "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.", 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.', 'Oil prices soar to all-time record, posing new menace to US economy

In [25]:
import ollama
from sklearn.metrics import accuracy_score
# Function to interact with the Ollama API
def chat_with_context(history):
    response = ollama.chat(model='llama3', messages=history)
    return response['message']['content']

# Initialize the chat history
chat_history = []
num_samples = 5

# Step 1: Define Task Description for AG News Dataset
task_description = "We need to predict the category of a given news article. The labels are 'World', 'Sports', 'Business', and 'Sci/Tech'."
instruction = "Classify each news article as 'World', 'Sports', 'Business', or 'Sci/Tech'."

generate_samples_request = (
    f"As an advanced language model you should create {num_samples} samples for the task outlined below.\n"
    "Generate samples that are likely to be correctly classified as 'World', 'Sports', 'Business', or 'Sci/Tech' and samples that might be misclassified according to the task instructions.\n\n"
    f"### Task Description:\n{task_description}\n\n"
    f"### Task Instructions:\n{instruction}\n\n"
    "### Requirements for Samples:\n"
    "1. Each sample must present a unique and intricate challenge.\n"
    "2. The complexity of the samples should be such that simply applying the given task instruction would likely lead to incorrect or incomplete results for some samples.\n"
    "3. The samples should cover a diverse range of scenarios within the scope of the task, avoiding repetition and predictability.\n"
    "4. Ensure that the samples, while challenging, remain realistic and pertinent to the task's context.\n"
    "Generate the samples keeping these requirements in mind.\n###"
)

chat_history.append({'role': 'user', 'content': task_description})
chat_history.append({'role': 'user', 'content': generate_samples_request})

samples_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': samples_response})
print("Generated Samples:")
print(samples_response)

# Step 2: Analyze Samples with Chain of Thought
analyze_samples_request = (
    f"Here are some samples: {samples_response}\nUsing chain of thought, analyze these samples "
    "and conclude a procedure for predicting whether a news article is 'World', 'Sports', 'Business', or 'Sci/Tech'. Identify key characteristics of both correctly and incorrectly classified samples, capture the mistakes from failed cases, and conclude under what circumstances we should predict each label."
)

chat_history.append({'role': 'user', 'content': analyze_samples_request})

analysis_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': analysis_response})
print("Analysis of Samples:")
print(analysis_response)

# Step 3: Generate Optimized Prompt
generate_prompt_request = (
    f"Based on the following analysis: {analysis_response}\nGenerate an optimized prompt for predicting "
    "whether a news article is 'World', 'Sports', 'Business', or 'Sci/Tech'. Ensure the model responds only with 'World', 'Sports', 'Business', or 'Sci/Tech'.\n\n"
    "### Requirements for Optimized Prompt:\n"
    "1. The prompt must include a clear description of the task and the labels.\n"
    "2. It should provide a comprehensive criteria for classifying news articles as 'World', 'Sports', 'Business', or 'Sci/Tech' based on the analysis.\n"
    "3. The prompt must ensure that the model responds strictly with 'World', 'Sports', 'Business', or 'Sci/Tech'.\n"
    "4. The prompt should help the model avoid common pitfalls and misclassifications identified during the analysis.\n"
    "5. Ensure the language is unambiguous and tailored to maximize the model's prediction accuracy."
    "6. Encourage the model to think step by step."
    "Respond with no other explanation but only the content of the prompt that is ready for the model to predict\n"
    "Prompt:"
)

chat_history.append({'role': 'user', 'content': generate_prompt_request})

optimized_prompt_response = chat_with_context(chat_history)
chat_history.append({'role': 'assistant', 'content': optimized_prompt_response})
print("Optimized Prompt:")
print(optimized_prompt_response)

# Function to predict the label of a given news article
def get_prediction(text):
    prompt = (
        f"{optimized_prompt_response}\n\n"
        f"article:{text}\n"
        "### Requirements:\n"
        "1. Respond with only the label name ('World', 'Sports', 'Business', or 'Sci/Tech').\n"
        "2. Do not provide any additional text or explanation.\n"
        "Respond with only the label name:"
    )
    response = ollama.generate(model='llama3', prompt=prompt)

    # Extract the predicted label from the response
    prediction = response['response'].strip().replace("**", "").replace("'", "").replace('"', '')
    if prediction not in ['World', 'Sports', 'Business', 'Sci/Tech']:
        prediction = "Invalid Prediction"
    
    # format the prediction so that if it keeps only 'World', 'Sports', 'Business', or 'Sci/Tech' as output
    return prediction

predictions = []
# Make predictions
for i, text in enumerate(texts):
    prediction = get_prediction(text)
    if i < 3:
        print(f"Predicted Label: {prediction}\n")
    if i % 1000 == 0:
        print(f"Predicted {i} samples out of {len(texts)}")
    if i == 1000:
        break
    predictions.append(prediction)

# Step 4: Evaluate the Model
true_labels = [mapping[label] for label in labels[:len(predictions)]]
accuracy = accuracy_score(true_labels, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Generated Samples:
Here are five news article samples with varying levels of complexity and challenge:

**Sample 1:**
Title: "Record-Breaking Heat Wave Hits Europe"
Text: A severe heat wave has swept across Europe, breaking temperature records and causing widespread disruptions. The extreme weather has led to power outages, wildfires, and health concerns.

**Classification:** 'World' (likely correct)

This sample is straightforward, as the article discusses a global weather event that affects multiple countries. It's a clear candidate for the 'World' category.

**Sample 2:**
Title: "NBA Star Injured in Car Accident; Team Rallies Behind Him"
Text: Professional basketball player James Johnson was involved in a serious car accident, leaving him with severe injuries. His teammates have rallied around him, and his fans are sending well-wishes for his recovery.

**Classification:** 'Sports' (likely correct)

This sample is also straightforward, as it's an article about a sports-related incid

# Basic Prompt

In [24]:
# Function to predict the label of a given news article
def get_prediction(text):
    prompt = (
        f"Predict the news article into one of following categories: ('World', 'Sports', 'Business', or 'Sci/Tech')\n\n"
        f"article:{text}\n"
        "### Requirements:\n"
        "1. Respond with only the label name ('World', 'Sports', 'Business', or 'Sci/Tech').\n"
        "2. Do not provide any additional text or explanation.\n"
        "Respond with only the label name:"
    )
    response = ollama.generate(model='llama3', prompt=prompt)

    # Extract the predicted label from the response
    prediction = response['response'].strip().replace("**", "").replace("'", "").replace('"', '')
    if prediction not in ['World', 'Sports', 'Business', 'Sci/Tech']:
        prediction = "Invalid Prediction"
    
    # format the prediction so that if it keeps only 'World', 'Sports', 'Business', or 'Sci/Tech' as output
    return prediction

predictions = []
# Make predictions
for i, text in enumerate(texts):
    prediction = get_prediction(text)
    if i < 3:
        print(f"Predicted Label: {prediction}\n")
    if i % 1000 == 0:
        print(f"Predicted {i+1} samples out of {len(texts)}")
    if i == 1000:
        break
    predictions.append(prediction)

# Step 4: Evaluate the Model
true_labels = [mapping[label] for label in labels[:len(predictions)]]
accuracy = accuracy_score(true_labels, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Predicted Label: Business

Predicted 1 samples out of 120000
Predicted Label: Business

Predicted Label: Business

Predicted 1001 samples out of 120000
Model Accuracy: 0.68
